# System Public API Examples

This notebook provides basic examples of how to use System's public API. Full documentation is available on our [API documentation site](https://systeminternal.github.io/api/).

Running these examples requires you to have an API key, provided by System. Please contact us at [api@system.com](mailto:api@system.com) if you are interested in exploring our public API.

## 1. Import libraries and define helper functions

In [ ]:
!pip install api-client==1.22.0 --extra-index-url https://pypi.fury.io/systeminc/
!pip install pandas

In [5]:
import api_client
import pandas as pd
import json
import os

In [6]:
def pprint_finding(f):
    print(f"{f.variable_1_name} - ({f.statistic_type}: {f.statistic_value}) -> {f.variable_2_name}")

def pprint_topic(t):
    print(f"{t.name} ({t.topic_id})")
    
_FINDING_COLUMNS = ['id', 'variable_1', 'statistic_type', 'statistic_value', 'ci_upper', 'ci_lower', 'p_value']

## 2. Create a service client and initialize endpoints

In [ ]:
%env YOUR_API_KEY=<your-api-key>

In [8]:
API_KEY = os.environ.get("YOUR_API_KEY")
system_client = api_client.ApiClient(header_name="x-api-key", header_value=API_KEY)

topics_api = api_client.api.topics_api.TopicsApi(system_client)
relationships_api = api_client.api.relationships_api.RelationshipsApi(system_client)
studies_api = api_client.api.studies_api.StudiesApi(system_client)
variables_api = api_client.api.variables_api.VariablesApi(system_client)
findings_api = api_client.api.statistical_findings_api.StatisticalFindingsApi(system_client)

## 3. Search for and retrieve a topic

In [9]:
def search_topic_by_name(topic_name: str):
  print(f"Search for a topic by name ({topic_name}): ")
  fetched_topics = topics_api.get_topics(search=f"name:{topic_name}")
  topics_json = [json.loads(topic.json()) for topic in fetched_topics.data]
  df = pd.DataFrame(topics_json)
  print(df)
  print("")
  return df['id'][3]

In [11]:
topic_id = search_topic_by_name("osteoporosis")

Search for a topic by name (osteoporosis): 
           id                                name wikidata_id   umls_cuis  \
0  kELO0u-WL3  osteoporosis-pseudoglioma syndrome   Q32136402  [C0432252]   
1  ZwLPJsQaKE         postmenopausal osteoporosis   Q55950294  [C0029458]   
2  TzD7fS4xV9    idiopathic juvenile osteoporosis    Q6318965  [C0264080]   
3  YCCUBPRxoV                        osteoporosis     Q165328  [C0029456]   

                            category                          roles  
0  Biomedical - Diseases & disorders  [Determinant, Outcome, Agent]  
1  Biomedical - Diseases & disorders  [Determinant, Outcome, Agent]  
2  Biomedical - Diseases & disorders  [Determinant, Outcome, Agent]  
3  Biomedical - Diseases & disorders  [Determinant, Outcome, Agent]  



In [12]:
def search_topic_by_filter(wikidata_id: str):
  print(f"Filter topics by Wikidata ID ({wikidata_id}): ")
  fetched_topics = topics_api.get_topics(filter=f"wikidata_id:{wikidata_id}")
  topics_json = [json.loads(topic.json()) for topic in fetched_topics.data]
  df = pd.DataFrame(topics_json)
  print(df)
  print("")
  return df['id'][0]

In [15]:
topic_id_from_filter = search_topic_by_filter("Q55950294")

Filter topics by Wikidata ID (Q55950294): 
           id                         name wikidata_id   umls_cuis  \
0  ZwLPJsQaKE  postmenopausal osteoporosis   Q55950294  [C0029458]   

                            category                          roles  
0  Biomedical - Diseases & disorders  [Determinant, Outcome, Agent]  



In [16]:
def get_topic_by_id(topic_id: str):
  print("Get a topic directly by id: ")
  fetched_topic = topics_api.get_topic_by_id(topic_id)
  topic_json = json.loads(fetched_topic.json())
  df = pd.DataFrame([topic_json])
  print(df)
  print("")

In [18]:
get_topic_by_id(topic_id)

Get a topic directly by id: 
           id          name wikidata_id   umls_cuis  \
0  YCCUBPRxoV  osteoporosis     Q165328  [C0029456]   

                            category                          roles  
0  Biomedical - Diseases & disorders  [Determinant, Outcome, Agent]  



## 4. Retrieve variables and determinants associated with a topic

In [20]:
def get_variables_by_topic_id(topic_id: str):
  print("Get variables by topic ID: ")
  fetched_variables = variables_api.get_variables(filter=f"topic.id:{topic_id}", limit=25)
  variables_json = [json.loads(variable.json()) for variable in fetched_variables.data]
  df = pd.DataFrame(variables_json)
  print(df)
  print("")
  return ""

In [ ]:
variable_id = get_variables_by_topic_id(topic_id)

In [23]:
def get_determinants_by_topic_id(topic_id: str):
  print("Get determinants by topic ID: ")
  fetched_determinants = topics_api.get_topic_determinants(topic_id, filter="num_significant_findings:>1", sort="median_effect_size:desc")
  determinants_json = [json.loads(determinant.json()) for determinant in fetched_determinants.data]
  df = pd.DataFrame(determinants_json)
  column_order = ['id',  'topic',  'num_findings', 'num_studies', 'num_significant_findings', 'highest_cited', 'median_effect_size', 'last_updated', 'has_experimental_trial']
  df = df[column_order]
  print(df)
  print("")
  return df['id'][1]

In [25]:
relationship_id = get_determinants_by_topic_id(topic_id)

Get determinants by topic ID: 
                      id                                              topic  \
0  MPr4sIFHU9|YCCUBPRxoV  {'id': 'MPr4sIFHU9', 'name': 'allele', 'wikida...   
1  ELjr0Yjdsc|YCCUBPRxoV  {'id': 'ELjr0Yjdsc', 'name': 'postmenopause', ...   
2  wWbfsY5E0d|YCCUBPRxoV  {'id': 'wWbfsY5E0d', 'name': 'sex', 'wikidata_...   
3  HejFSI-X_z|YCCUBPRxoV  {'id': 'HejFSI-X_z', 'name': 'age group', 'wik...   
4  EhpdS79G1u|YCCUBPRxoV  {'id': 'EhpdS79G1u', 'name': 'ethnicity and ra...   
5  W_5xNoEZX9|YCCUBPRxoV  {'id': 'W_5xNoEZX9', 'name': 'menarche', 'wiki...   
6  -thTZiXKag|YCCUBPRxoV  {'id': '-thTZiXKag', 'name': 'genotype', 'wiki...   
7  i4pV-bfaG8|YCCUBPRxoV  {'id': 'i4pV-bfaG8', 'name': 'fibroblast growt...   
8  MIpYYKT5zF|YCCUBPRxoV  {'id': 'MIpYYKT5zF', 'name': 'haplotype', 'wik...   
9  D525NDqU1Y|YCCUBPRxoV  {'id': 'D525NDqU1Y', 'name': 'genetic polymorp...   

   num_findings  num_studies  num_significant_findings  highest_cited  \
0             2           

## Search for and retrieve a study

In [28]:
def get_study_by_doi(doi: str):
	print("Get study by DOI: ")
	fetched_studies = studies_api.get_studies(filter=f"doi:{doi}")
	studies_json = [json.loads(study.json()) for study in fetched_studies.data]
	df = pd.DataFrame(studies_json)
	print(df)
	print("")
	return df['id'][0]

In [29]:
study_id = get_study_by_doi("10.1016/j.bone.2010.03.017")

Get study by DOI: 
           id                         doi  \
0  BwNHkghmPt  10.1016/j.bone.2010.03.017   

                                               title  \
0  Prevalence and risk factors of osteoporosis in...   

                                             authors journal publish_date  \
0  [{'full_name': 'Chan Soo Shin', 'orcid': '0000...    Bone   2010-08-01   

  population  cited_by retraction_reason  
0       None       117              None  



## 6. Retrieve statistical findings from relationships or studies

In [30]:
def get_findings_by_relationship_id(relationship_id: str):
  print("Get findings by relationship ID: ")
  fetched_findings = relationships_api.get_relationship_statistical_findings(relationship_id)
  findings_json = [json.loads(finding.json()) for finding in fetched_findings.data]
  df = pd.DataFrame(findings_json)
  df = df[_FINDING_COLUMNS]
  print(df)
  print("")
  return df['id'][1]

In [32]:
finding_from_relationship = get_findings_by_relationship_id("ELjr0Yjdsc|YCCUBPRxoV")

Get findings by relationship ID: 
           id                                         variable_1  \
0  GjA1LF3Lj9  {'id': '3V4swOcx1l', 'name': 'Postmenopausal R...   
1  E_BM4j2unD  {'id': '-ZCF1D2ouM', 'name': 'post-menopause s...   
2  ye22t7nhbh  {'id': 'Pl93A9-O_D', 'name': 'postmenopausal s...   
3  aGx8oVkn2o  {'id': 'VQVyc1mbWS', 'name': 'Postmenopausal w...   
4  LtBL-kaCCf  {'id': 'C5UOcrWu3g', 'name': 'postmenopausal w...   
5  B1OQ2yY7fx  {'id': 'VQVyc1mbWS', 'name': 'Postmenopausal w...   
6  AAdXio0yX6  {'id': 'eLsyYFQuow', 'name': 'Post-menopausal ...   
7  lyKucekQM5  {'id': 'cdraWaZ_On', 'name': 'postmenopausal f...   
8  nZRejgEftk  {'id': 'CKLH8oolgJ', 'name': 'Postmenopausal w...   
9  iYgCcdLnz2  {'id': '5IzHJFKc9Q', 'name': 'postmenopausal w...   

                   statistic_type  statistic_value  ci_upper  ci_lower  \
0  mean_difference_unstandardized            9.600       NaN       NaN   
1                      odds_ratio            7.320    17.600     3.05

In [36]:
def get_findings_by_study_id(study_id: str):
	print("Get findings by study ID: ")
	fetched_findings = studies_api.get_study_statistical_findings(study_id)
	findings_json = [json.loads(finding.json()) for finding in fetched_findings.data]
	df = pd.DataFrame(findings_json)
	df = df[_FINDING_COLUMNS]
	print(df)
	print("")
	return df['id'][0]

In [37]:
finding_from_study = get_findings_by_study_id(study_id)

Get findings by study ID: 
           id                                         variable_1  \
0  UgW-Nd8K5c  {'id': 'KWPYSf516x', 'name': 'having more than...   
1  C6TEWl_9DL  {'id': 'ikQSAGll0n', 'name': 'menarche after a...   
2  E_BM4j2unD  {'id': '-ZCF1D2ouM', 'name': 'post-menopause s...   
3  hH8_zLfcjv  {'id': 'hazlzJQfcm', 'name': 'last delivery af...   
4  VzaKeg1O7Q  {'id': 'ZSRCgqrjON', 'name': 'more than 17 yea...   
5  EZnBmNHRv6  {'id': 'd4EXjJcR2n', 'name': 'smoking> or =1 p...   
6  Z_pdKSy6EH  {'id': 'XatSLZcfba', 'name': 'past fracture hi...   
7  lx6EfFujo0  {'id': 'R7CR1MHpOA', 'name': 'calcium intake> ...   
8  p9OFTXZYY6  {'id': 'dIxvV7CyDq', 'name': 'monthly income a...   
9  1G5qAWF7ng   {'id': 'ds-IcQ6tDc', 'name': 'college graduate'}   

  statistic_type  statistic_value  ci_upper  ci_lower  p_value  
0     odds_ratio             1.42      1.89      1.07    0.001  
1     odds_ratio             1.46      1.87      1.14    0.001  
2     odds_ratio             